In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.stats as st
from matplotlib.ticker import FuncFormatter
import scipy.optimize as op
from preprocessing import *
plt.style.use("dark_background") # Config plots for dark mode, delete if on light mode
plt.rcParams['figure.dpi'] = 150 # Hi-res plots

In [3]:
def N(t_scalar, t):
    """
    Returns the number of times in t less than or equal to t_scalar.
    Is used to compute N(t_{i,k}) and N'(t_{i,k}) depending on whether t above is t or t_prime
    """
    return np.searchsorted(t, t_scalar, side="right")

def getTimeDifferences(t, t_prime):
    """
    Input: (sorted) times for a particular station i
    Output: List of differences indexed by [h][k] for this station i
    """

    # h goes until N(t[-1], t) assuming T = t[-1]
    T = end_T
    D_result = []
    for h in range(1, N(T, t)+1):
        differences_list = []
        # Construct list of t_ih - t'_ik for k = 1 to N'(T)
        differences_list.append(t[h-1] - t_prime[N(t[h-2], t_prime):N(t[h-1], t_prime)])

        D_result.append(np.array(differences_list))

    return D_result

getTimeDifferences(np.array([1,2,3]),np.array([1.1,2.2,3.3]))

[array([], shape=(1, 0), dtype=float64), array([[0.9]]), array([[0.8]])]

In [22]:
durations_dict = {}
for st_id in train_sorted_start_ids:
    station_train_data = train_bike_data[train_bike_data.end_id == st_id]
    durations_dict[st_id] = station_train_data.duration.to_numpy()

In [23]:
def compensator_m5(t_scalar, t, t_prime, durations_i, alpha_i, beta_i, beta_i_prime,delta_i, delta_i_prime, gamma_i, lambda_i):
    # Durations in minutes
    """
    t_scalar: scalar value where Lambda_i(t) is to be evaluated
    t_prime: list of arrival times at station i

    NOTE: t_prime NEEDS TO BE SORTED HERE.
    """
    

    term1 = lambda_i * t_scalar
    term2 = -(alpha_i / beta_i_prime) * np.sum(np.exp(-beta_i_prime * (t_scalar - t_prime[:N(t_scalar, t_prime)]))*np.exp(-beta_i(durations_i - 1))-1)
    term3 = -(gamma_i / delta_i_prime) * np.sum(np.exp(-delta_i_prime * (t_scalar - t[:N(t_scalar, t_prime)]))*np.exp(-delta_i*np.abs(durations_i - 30))-1)
    return term1 + term2 + term3


In [ ]:
def genB_i(h, durations_i, beta_i, beta_prime,)